In [4]:
import pandas as pd
import requests as rqs
import os
import json
import pymongo
import datetime
import plotly.express as px

year = 2020
month = 2

myclient = pymongo.MongoClient(f'mongodb://{os.getenv("MONGO_URL","192.168.2.1")}:27017',username=os.getenv("MONGO_USERNAME","hterra"),password=os.getenv("MONGO_PASSWORD","terrac0t@"), unicode_decode_error_handler='ignore')
data = pd.DataFrame(list( myclient["activitywatch"]["aw-watcher-window"].find({'year': year,'month':month,'duration': {'$gt': 1},'app': {'$regex':'.exe', '$options':'i'}})))
data.set_index("_id", inplace=True)
data['app'] = data['app'].str.upper()
data['app'] = data['app'].str.replace(".EXE","")
host = pd.DataFrame(list( myclient["activitywatch"]["host"].find()))
data = pd.merge(data, host, how='left')
data.set_index("_id", inplace=True)
myclient.close()                            
data = data.loc[data['app']!= "LOCKAPP"]
data = data.loc[data['app']!= "UNKNOWN"]
                               

In [10]:
data_apps_global = data
data_apps_global = data_apps_global.groupby(['day'], as_index=False)['duration'].sum()
data_apps_global.index = data_apps_global['day']
data_apps_global = data_apps_global.reindex(range(1,28), fill_value=0)
data_apps_global['date'] = data_apps_global.index
data_apps_global['duration'] = (data_apps_global['duration']/3600)/21
fig = px.bar(data_apps_global, x='date', y='duration',color='duration',
             labels={'date':'Fecha','duration':'Horas laboradas promedio'}, height=300)
fig.show()

In [11]:
data_apps_global = data
data_apps_global = data_apps_global.loc[data_apps_global['app'].isin([
    'EXCEL','OUTLOOK','CHROME','EXPLORER','WINWORD','VISIO','LL300','TEAMS','NITROPDF','IEXPLORER'
])]
data_apps_global = data_apps_global.groupby(['hour'], as_index=False)['duration'].sum()
data_apps_global['timestr'] = data_apps_global['duration'].apply(lambda x:
    '{:.0f} hr {:.0f} min {:.0f} secs'.format(
        divmod(x,60*60)[0],
        *divmod(divmod(x,60*60)[1],60))
                                 )
fig = px.line(data_apps_global, x="hour", y="duration", height=300)
fig.show()
data_apps_global

,hour,duration,timestr
0,0,84193.400,23 hr 23 min 13 secs
1,1,7.845,0 hr 0 min 8 secs
2,6,1044.959,0 hr 17 min 25 secs
3,13,17651.045,4 hr 54 min 11 secs
4,14,176469.837,49 hr 1 min 10 secs
5,15,441639.755,122 hr 40 min 40 secs
6,16,521831.149,144 hr 57 min 11 secs
7,17,509369.963,141 hr 29 min 30 secs
8,18,522963.528,145 hr 16 min 4 secs
9,19,508437.513,141 hr 13 min 58 secs


In [10]:
data_apps_global = data
data_apps_global['time'] = data_apps_global['timestamp'].dt.strftime('%H')
data_apps_global = data_apps_global.loc[data_apps_global['app'].isin([
    'EXCEL','OUTLOOK','CHROME','EXPLORER','WINWORD','VISIO','LL300','TEAMS','NITROPDF','IEXPLORER'
])]
data_apps_global.loc[data_apps_global['time'].astype(int) > 22]

,timestamp,duration,host,app,title,user,date,time
_id,,,,,,,,
5e5830f54185a9317a7400e6,2020-02-27 23:56:42.649,297.511,DESKTOP-RIEHMGE,CHROME,Untitled - Jupyter Notebook - Google Chrome,nmartinez,2020-02-27,23
5e5830f54185a9317a7400e6,2020-02-27 23:53:19.303,195.819,DESKTOP-RIEHMGE,CHROME,Untitled - Jupyter Notebook - Google Chrome,nmartinez,2020-02-27,23
5e5830f54185a9317a7400e6,2020-02-27 23:48:04.689,308.159,DESKTOP-RIEHMGE,CHROME,Untitled - Jupyter Notebook - Google Chrome,nmartinez,2020-02-27,23
5e5830f54185a9317a7400e6,2020-02-27 23:47:44.406,17.072,DESKTOP-RIEHMGE,CHROME,python - Matplotlib: How to change figsize for...,nmartinez,2020-02-27,23
5e5830f54185a9317a7400e6,2020-02-27 23:47:39.046,4.283,DESKTOP-RIEHMGE,CHROME,plot.bar size - Buscar con Google - Google Chrome,nmartinez,2020-02-27,23
...,...,...,...,...,...,...,...,...
5e5830f54185a9317a7400e4,2020-02-05 22:54:05.310,7.629,DESKTOP-PDQAQA7,CHROME,FileMaker WebDirect - Google Chrome,mvaldez,2020-02-05,22
5e5830f54185a9317a7400e4,2020-02-05 22:52:47.218,76.948,DESKTOP-PDQAQA7,EXCEL,423 FINAL - Excel,mvaldez,2020-02-05,22
5e5830f54185a9317a7400e4,2020-02-05 22:51:46.800,22.014,DESKTOP-PDQAQA7,EXCEL,423 FINAL - Excel,mvaldez,2020-02-05,22


In [106]:
data_apps_global = data
data_apps_global['date'] = data_apps_global['timestamp'].dt.strftime('%Y-%m-%d')
data_apps_global = data_apps_global.groupby(['app'], as_index=False)['duration'].sum()
data_apps_global = data_apps_global.loc[~data_apps_global['app'].isin([
    'APPLICATIONFRAMEHOST','CODE','SSMS','MSTSC','SHELLEXPERIENCEHOST','POSTMAN','STARUML','WINDOWSTERMINAL','TEAMVIEWER',
    'POWERSHELL','DEVENV','TASKMGR','WINPROJ','SNIPPINGTOOL','MUSNOTIFICATIONUX','PICKERHOST','DWM','SEARCHUI','DSTERMSERV',
    'CMD','CREDENTIALUIBROKER','STARTMENUEXPERIENCEHOST','MONGODBCOMPASS','PSAWIZRD','SPLWOW64','ATT','JP2LAUNCHER','ANTITWIN',
    'MSC','JAVAW','CSRSS'
])]
data_apps_global = data_apps_global.sort_values('duration', ascending=False)[0:10]
data_apps_global['duration'] = data_apps_global['duration']/21
data_apps_global['time'] = data_apps_global['duration'].apply(lambda x:
    '{:.0f} hr {:.0f} min {:.0f} secs'.format(
        divmod(x,60*60)[0],
        *divmod(divmod(x,60*60)[1],60))
                                 )
fig = px.pie(data_apps_global, values='duration', names='app', title='10 aplicaciónes mas usasdas')
fig.show()

data_apps_global

,app,duration,time
28,EXCEL,73193.161571,20 hr 19 min 53 secs
76,OUTLOOK,43656.725619,12 hr 7 min 37 secs
8,CHROME,38793.411143,10 hr 46 min 33 secs
29,EXPLORER,22040.063714,6 hr 7 min 20 secs
117,WINWORD,13141.485286,3 hr 39 min 1 secs
112,VISIO,5095.994762,1 hr 24 min 56 secs
50,LL300,4699.282095,1 hr 18 min 19 secs
106,TEAMS,2742.801476,0 hr 45 min 43 secs
66,NITROPDF,2525.541333,0 hr 42 min 6 secs
40,IEXPLORE,2310.054095,0 hr 38 min 30 secs


In [5]:
data

,duration,host,app,title,year,month,day,hour,minute,time,user
_id,,,,,,,,,,,
5e5830f54185a9317a7400e6,302.283,DESKTOP-RIEHMGE,CHROME,Untitled - Jupyter Notebook - Google Chrome,2020,2,28,22,42,0 hr 5 min 2 secs,nmartinez
5e5830f54185a9317a7400e6,312.332,DESKTOP-RIEHMGE,CHROME,Portainer - Google Chrome,2020,2,28,22,37,0 hr 5 min 12 secs,nmartinez
5e5830f54185a9317a7400e6,1.059,DESKTOP-RIEHMGE,CHROME,matplotlib.pyplot.pie — Matplotlib 3.1.3 docum...,2020,2,28,22,37,0 hr 0 min 1 secs,nmartinez
5e5830f54185a9317a7400e6,3.165,DESKTOP-RIEHMGE,CHROME,wPmlihGCcUq7gAAAABJRU5ErkJggg== (1143×319) - G...,2020,2,28,22,36,0 hr 0 min 3 secs,nmartinez
5e5830f54185a9317a7400e6,3.167,DESKTOP-RIEHMGE,CHROME,Untitled - Jupyter Notebook - Google Chrome,2020,2,28,22,36,0 hr 0 min 3 secs,nmartinez
...,...,...,...,...,...,...,...,...,...,...,...
5e5830f54185a9317a7400e4,58.139,DESKTOP-PDQAQA7,CHROME,Outlook: calendario y correo electrónico perso...,2020,2,5,14,11,0 hr 0 min 58 secs,mvaldez
5e5830f54185a9317a7400e4,1.095,DESKTOP-PDQAQA7,CHROME,Nueva pestaña - Google Chrome,2020,2,5,14,11,0 hr 0 min 1 secs,mvaldez
5e5830f54185a9317a7400e4,1.165,DESKTOP-PDQAQA7,OUTLOOK,RV: Reportes Semanales Firma Car 02-02-2020 - ...,2020,2,5,14,11,0 hr 0 min 1 secs,mvaldez
